In [ ]:
import numpy as np
import pandas as pd
class Node:
  def __init__(self,class_name,condition):
    self.class_name = class_name # column name eg : Age
    self.condition = condition # value of that column eg : >21
    self.children = []
  
  def getId(self):
    return self.class_name + " " + self.condition # Age >21

In [ ]:
class DecisionTree:
  def __init__(self,dataset,target_class):
    self.root = Node("Root node","has no condition") 
    self.dataset = dataset
    self.target_class = target_class
    #max information gain classes that are used by the constructed tree for classifying 
    self.max_gain_classes = [] 
  
  def get_entropy(self,column,dataset,p,n):
    categories = {}
    for entry in list(dataset[column]):
      categories[entry] = None

    entropy = 0
    for entry in list(categories.keys()):
      df = dataset[dataset[column]==entry]
      p_df,n_df,I_pn = self.get_info_gain(list(df[self.target_class]))
      entropy+=((p_df+n_df)/(p+n))*I_pn
    return entropy

  
  def get_info_gain(self,column):
    categories = {}
    for entry in column:
      categories[entry] = categories[entry] + 1 if entry in categories else 1
    keys = list(categories.keys())
    p = categories["Y"]/len(column) if "Y" in categories else 0
    n = categories["N"]/len(column) if "N" in categories else 0
    P_ratio = p/(n+p)
    N_ratio = n/(n+p)
    if(P_ratio==0):
      return p,n,-1*(N_ratio*np.log2(N_ratio))
    elif(N_ratio==0):
      return p,n,-1*(P_ratio*np.log2(P_ratio))
    return p,n,-1*((P_ratio*np.log2(P_ratio)) + (N_ratio*np.log2(N_ratio)))
  
  def build(self,node,dataset):
    if node is None:
      node = self.root

    #node in which corresponding dataset has same values in target column has it's child as leaf node
    target_column = list(dataset[self.target_class])
    p,n,target_column_info_gain = self.get_info_gain(target_column)
    if target_column_info_gain==0: 
      node.children.append(Node(self.target_class,"Y" if n==0 else "N"))
      return node

    max_gain = 0
    class_Indexes = {}
    columns = list(dataset.columns) 
    max_gain_class = columns[0]

    #calculate entropy and information gain for each column and select column which has highest information gain
    for index in range(0,len(columns)):
      if(columns[index]==self.target_class):
        continue
      column_entropy = self.get_entropy(columns[index],dataset,p,n)
      class_Indexes[columns[index]] = index 
      gain = target_column_info_gain - column_entropy
      if(gain > max_gain):
        max_gain = gain
        max_gain_class = columns[index]
      self.max_gain_classes.append(max_gain_class)

    # generate seperate datasets for each value of max_gain_class
    class_Map = {}
    records = list(dataset.to_records(index=False))
    for row in records:
      if row[class_Indexes[max_gain_class]] in class_Map:
        class_Map[row[class_Indexes[max_gain_class]]].append(tuple(row))  
      else : 
        class_Map[row[class_Indexes[max_gain_class]]]=[tuple(row)]

    for key in class_Map:
      df_child = pd.DataFrame(list(class_Map[key]),columns = columns)
      df_child.drop(columns=[max_gain_class]) # remove the redundant max_gain_class column from the child dataset
      child = self.build(Node(max_gain_class,key),df_child) # get remaining subtree with the given child as it's root
      node.children.append(child)

    return node


In [ ]:
training_data = [
  ['<21', 'High', 'M', 'Single', 'N'],
  ['<21', 'High', 'M', 'Married', 'N'],
  ['21-35', 'High', 'M', 'Single', 'Y'],
  ['>35', 'Medium', 'M', 'Single', 'Y'],
  ['>35', 'Low', 'F', 'Single', 'Y'],
  ['>35', 'Low', 'F', 'Married', 'N'],
  ['21-35', 'Low', 'F', 'Married', 'Y'],
  ['<21', 'Medium', 'M', 'Single', 'N'],
  ['<21', 'Low', 'F', 'Married', 'Y'],
  ['>35', 'Medium', 'F', 'Single', 'Y'],
  ['<21', 'Medium', 'F', 'Married', 'Y'],
  ['21-35', 'Medium', 'M', 'Married', 'Y'],
  ['21-35', 'High', 'F', 'Single', 'Y'],
  ['>35', 'Medium', 'M', 'Married', 'N']
]
header = ["Age", "Income","Gender","Marital Status","Buys"]
dataset = pd.DataFrame(training_data,columns=header)
dt = DecisionTree(dataset,"Buys")
root = dt.build(None,dataset)

In [ ]:
#generate IF-THEN rules from the constructed tree
rules = []
def buildRules(root,rule_string,target_class):
  if(len(root.children)==0):
    rules.append(rule_string)
    return
  for node in root.children:
    if(node.class_name==target_class):
      buildRules(node,rule_string + " THEN " + node.getId(),target_class)
    else :
      buildRules(node,rule_string + " IF " + node.getId(),target_class)

buildRules(root,"","Buys")
for rule in rules:
  print(rule)

 IF Age <21 IF Gender M THEN Buys N
 IF Age <21 IF Gender F THEN Buys Y
 IF Age 21-35 THEN Buys Y
 IF Age >35 IF Marital Status Single THEN Buys Y
 IF Age >35 IF Marital Status Married THEN Buys N


In [ ]:
def bfs(root,data):
  child_list=[root]
  index=0
  while(len(child_list)!=0):
    node = child_list[0]
    for child in node.children:
      if(child.condition in ['Y','N']):
        return child.condition
      if(data[index]==child.condition):
        child_list.append(child)
        index+=1
    child_list.remove(node)
  return None

#prediction by level wise search of queries and reaching upto leaf node
def predict(data,root,classes,columns):
  #remove the classes which are not in max information gain classes list
  for index in range(0,len(data)) :
    if columns[index] not in classes:
      data.remove(data[index]) 
  return bfs(root,data)


testing_data = [
  ['<21', 'Low', 'F', 'Married']
]

ans = predict(testing_data[0],root,dt.max_gain_classes,header)
print(ans)

Y
